# BIOMAG introduction material 2. - Pytorch CNN basics

In [2]:
import torch # pytorch basic package
from torch import nn # neural net 
from torch.utils.data import DataLoader, Dataset # to work with data
from torchvision import datasets # built-in data
from torchvision.transforms import ToTensor # to convert nparrays/images into pytorch tensors
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from nn_utils import train, test

In [3]:
torch.manual_seed(0)

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [5]:
batch_size = 8

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_relu_stack = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=3, stride=1, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features = 64 * 7 * 7, out_features = 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

        self.flatten = nn.Flatten()


    def forward(self, x):
        x = self.conv_relu_stack(x)
        x = self.flatten(x)
    
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
# We create our model by instantiating the NeuralNetwork class and immediately move it to the GPU
model = CNNModel().to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

CNNModel(
  (conv_relu_stack): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3136, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


In [8]:
epochs = 5
train_losses = []
test_losses = []
for t in tqdm(range(epochs)):
    train_loss = train(train_dataloader, model, loss_fn, optimizer, 10000, device)
    test_loss = test(t, test_dataloader, model, loss_fn, device)

    train_losses.append(train_loss)
    test_losses.append(test_loss)

print("Done!")

  0%|          | 0/5 [00:00<?, ?it/s]

8/60000 : current_loss = 2.3110220432281494


 20%|██        | 1/5 [00:19<01:17, 19.40s/it]

End of epoch 1
 Accuracy: 75.99%, Average loss: 0.6779

8/60000 : current_loss = 1.2522040605545044


 40%|████      | 2/5 [00:36<00:54, 18.08s/it]

...

End of epoch 3
 Accuracy: 81.15%, Average loss: 0.5249

8/60000 : current_loss = 1.0463030338287354


 80%|████████  | 4/5 [01:11<00:17, 17.56s/it]

End of epoch 4
 Accuracy: 82.61%, Average loss: 0.4844

8/60000 : current_loss = 1.0195074081420898


100%|██████████| 5/5 [01:29<00:00, 17.89s/it]

End of epoch 5
 Accuracy: 83.36%, Average loss: 0.4555

Done!


## Homework
1. Try applying reqularization techniques to the network (add dropout to the end of the conv layer and reqular nn layers as well). What can you observe?
2. Until now, our networks outputted logits (a number between -inf and +inf), but we still got correct results because of the nn.CrossEntropyLoss (which does *something* in the background). If we were to do everything manually, we would have to apply a softmax layer in the end and use some different loss. Modify the architecture so that you do the probability counting manually as well.
3. Implement the LeNet network, one of the first neural networks for image classification (https://en.wikipedia.org/wiki/LeNet).